In [1]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [3]:
pip install pims_nd2

     ---------------------------------------- 0.0/19.7 MB ? eta -:--:--
     --------------------------------------- 0.0/19.7 MB 660.6 kB/s eta 0:00:30
     ---------------------------------------- 0.2/19.7 MB 2.4 MB/s eta 0:00:09
      --------------------------------------- 0.5/19.7 MB 3.5 MB/s eta 0:00:06
     - -------------------------------------- 0.7/19.7 MB 3.9 MB/s eta 0:00:05
     - -------------------------------------- 0.9/19.7 MB 4.0 MB/s eta 0:00:05
     -- ------------------------------------- 1.1/19.7 MB 4.1 MB/s eta 0:00:05
     -- ------------------------------------- 1.3/19.7 MB 4.1 MB/s eta 0:00:05
     --- ------------------------------------ 1.5/19.7 MB 4.2 MB/s eta 0:00:05
     --- ------------------------------------ 1.7/19.7 MB 4.3 MB/s eta 0:00:05
     ---- ----------------------------------- 2.0/19.7 MB 4.4 MB/s eta 0:00:05
     ---- ----------------------------------- 2.2/19.7 MB 4.4 MB/s eta 0:00:04
     ---- ----------------------------------- 2.4/19.7 MB 

In [4]:
import imageio  # make sure to install imageio with mamba install imageio
import pims_nd2
import nd2_to_caiman
import h5py
import numpy as np

In [5]:
nd2_path = "D:/sd_project_pbox/raw_data/OPI_2342/240318_baseline1/240318_OPI2342_baseline_1.nd2"
export_file = nd2_path[:-4] + "_exp.tiff"

## Get data as np array

In [6]:
nd2_file = pims_nd2.ND2_Reader(nd2_path)

# might need to save the metadata too
nd2_metadata = nd2_file.metadata

c:\Users\ttami\anaconda3\Lib\site-packages\pims\base_frames.py:478: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


In [7]:
export_arr = nd2_to_caiman.np_arr_from_nd2(nd2_path)  # FIXME: reading out results in same frames
export_arr.shape

(17978, 512, 512)

# TIFF with imageio: probably not multi-page.

In [8]:
imageio.mimwrite(export_file, export_arr)

error: argument out of range

In [9]:
# check if images are same
export_arr[0] == export_arr[100]

array([[ True,  True, False, ...,  True,  True,  True],
       [ True, False, False, ...,  True, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False]])

# HDF5 with h5py

In [10]:
h5_file = export_file[:-5] + ".h5"
print(h5_file)

D:/sd_project_pbox/raw_data/OPI_2342/240318_baseline1/240318_OPI2342_baseline_1_exp.h5


In [11]:
dataset_name = "mov"  # an annoying bug in CaImAn... passing 'var_name_hdf5': 'data' 
                      # to CNMFParams does not work. Stick to default name for now
with h5py.File(h5_file, 'w') as hf:
    dataset = hf.create_dataset("mov", shape=export_arr.shape, dtype=np.uint16)
    for i_frame in range(export_arr.shape[0]):
        dataset[i_frame] = export_arr[i_frame]

In [12]:
export_arr.shape[0]

17978

In [13]:
export_arr[0]

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [11, 15, 12, ..., 24, 17, 26],
       [34, 19, 22, ..., 25, 20, 28],
       [ 8, 18, 21, ..., 16,  6, 13]], dtype=uint16)

In [14]:
f = h5py.File(h5_file, "r")

In [15]:
f.get("data")